In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i_fnsd.cmd' % k)
solverF.init_grid()
solverF.init_flux_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFNSR%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, transient=True)
solverS.output_file = 'solid_outputs/resFNSR%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_robin_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=300.0)
abTempF2S = InterfaceData(size=inodes, value=300.0)
FluxS = InterfaceData(size=inodes, value=0.0)
FluxF = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=1e-6)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF2S = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)[f2s]

In [7]:
# coupling timing information, use fixed time step for now
Tfinal = 1.0
N = 100
dt = Tfinal/N
t = 0.0
avg_pc_iterations = 0
flog = open('FNSR%i.log' % k, mode='w')
flog.write('Fluid Neumann with solid Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FNSR%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(N):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        FluxF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        abTempF2S.curr[:] = abTempF.curr[f2s]
        # update solid robin interface
        solverS.update_robin_interface(
            tag=interface_tag,
            Ta=abTempF2S.curr,
            h_coeff=hF2S
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface flux
        FluxS.curr[:] = solverS.get_interface_flux(tag=interface_tag)
        
        # mapping
        FluxF.curr[:] = FluxS.curr[s2f]
        
        # update residual
        FluxF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(FluxF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            under_relax.determine_omega(FluxF)
            under_relax.update_solution(FluxF)
            solverF.put_heat_flux_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=FluxF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    solverS.write()
    fluid_ofile.write(solverF, t)
    avg_pc_iterations = avg_pc_iterations + pc_counts
flog.close()

step=0, pc_iterations=14.
step=1, pc_iterations=14.
step=2, pc_iterations=14.
step=3, pc_iterations=14.
step=4, pc_iterations=13.
step=5, pc_iterations=12.
step=6, pc_iterations=12.
step=7, pc_iterations=12.
step=8, pc_iterations=12.
step=9, pc_iterations=12.
step=10, pc_iterations=12.
step=11, pc_iterations=12.
step=12, pc_iterations=12.
step=13, pc_iterations=12.
step=14, pc_iterations=13.
step=15, pc_iterations=13.
step=16, pc_iterations=13.
step=17, pc_iterations=13.
step=18, pc_iterations=13.
step=19, pc_iterations=13.
step=20, pc_iterations=12.
step=21, pc_iterations=12.
step=22, pc_iterations=12.
step=23, pc_iterations=12.
step=24, pc_iterations=12.
step=25, pc_iterations=12.
step=26, pc_iterations=12.
step=27, pc_iterations=12.
step=28, pc_iterations=12.
step=29, pc_iterations=12.
step=30, pc_iterations=12.
step=31, pc_iterations=12.
step=32, pc_iterations=12.
step=33, pc_iterations=12.
step=34, pc_iterations=12.
step=35, pc_iterations=12.
step=36, pc_iterations=12.
step=37, pc

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat('numeric_res/kFNSR%i.mat' % k, {'theta': theta, 'avg_pc_its': avg_pc_iterations/N})

[0.85057386 0.90060101 0.91531867 0.91450607 0.91573515 0.91836917
 0.92038546 0.92219601 0.92455615 0.92578419 0.92736424 0.92855412
 0.93006253 0.93118044 0.93238135 0.93330865 0.93440068 0.93531656
 0.93610305 0.93699823 0.93771646 0.93845328 0.93913882 0.93973281
 0.94036517 0.94089274 0.94147593 0.94195048 0.9423427  0.94277736
 0.94320785 0.94355298 0.94392242 0.94424458 0.94455307 0.94481772
 0.94509723 0.94531963 0.94557882 0.94578855 0.94598049 0.9461345
 0.94629844 0.94645268 0.94659274 0.94671448 0.94683448 0.94693417
 0.94700874 0.94710126 0.94716432 0.94722461 0.94727006 0.94730939
 0.94732865 0.94736389 0.94736546 0.9473659  0.94736161 0.94734395
 0.94730079 0.94727478 0.94722061 0.94716818 0.94709229 0.94702128
 0.94691478 0.9468256  0.94669153 0.94655838 0.94644472 0.94628548
 0.94608244 0.94591592 0.9456954  0.94548397 0.94522951 0.94497054
 0.9446553  0.94436654 0.94399642 0.94362349 0.9432298  0.94280072
 0.94234027 0.9417374  0.94127694 0.94064351 0.9398906  0.93924